In [17]:
from helpers import load_CSD_data, make_element_wise_environments

In [18]:
#------loading subsample of dataset--------

PATH_TRAIN = "CSD-2k_relaxed_shifts.txt"
PATH_TEST = "CSD-500.txt"
#structures_test, shifts_test = load_CSD_data(PATH_TEST,random_subsample=100)
structures_train, shifts_train = load_CSD_data(PATH_TRAIN,random_subsample=100)

In [19]:
from rascal.representations import SphericalInvariants

hypers = {"soap_type": "PowerSpectrum",
          "interaction_cutoff": 3,
          "radial_basis": "GTO",
          "max_radial": 9,
          "max_angular": 9,
          "gaussian_sigma_constant": 0.3,
          "gaussian_sigma_type":"Constant",
          "cutoff_function_type":"ShiftedCosine",
          "cutoff_smooth_width": 0.5,
          "normalize": True,
          "compute_gradients":False,
          "cutoff_function_parameters":dict(rate=1,scale=3.5,exponent=4),
          #"optimization": dict(Spline=dict(accuracy=1.0e-05))
          #"expansion_by_species_method":'structure wise'
         }

In [20]:
calculator = SphericalInvariants(**hypers)

In [21]:
H_environments_train, H_shifts_train = make_element_wise_environments(calculator,structures_train,shifts_train,select=6)

In [6]:
print("%d megabytes" % (H_environments_train.size/1e06 * H_environments_train.itemsize))

1196 megabytes


In [22]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics.pairwise import polynomial_kernel

import numpy as np

y = H_shifts_train

X = H_environments_train
clf = KernelRidge(alpha=1e-06,kernel='poly', degree=2, coef0=1)
clf.fit(X, y)

KernelRidge(alpha=1e-06, degree=2, kernel='poly')

In [23]:
#eval
structures_test, shifts_test = load_CSD_data(PATH_TEST)
H_environments_test, H_shifts_test = make_element_wise_environments(calculator,structures_test,shifts_test,select=6)

In [24]:
y_predicted = clf.predict(H_environments_test)

In [17]:
y_predicted = []
for n, env in enumerate(H_environments_test):
    y_predicted.append(clf.predict(env.reshape(1, -1)))
    if n%50

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [27]:
np.sqrt(2*mean_squared_error(H_shifts_test[:,0], np.array(y_predicted),squared=True))

11.732554953455425

In [26]:
mean_absolute_error(H_shifts_test[:,0],y_predicted)

5.788674852147157

In [39]:
def optimize_hypers(X_train,X_test,y_train,y_test,y_ML_old,alphas=np.logspace(-8, 0, num=25)):
    errors = []
    errors_ml = []
    
    for val in alphas:
        clf = KernelRidge(alpha=val, degree=2)
        clf.fit(X_train, y_train)
        y_predicted = clf.predict(X_test)
        errors.append(np.sqrt(2*mean_squared_error(y_predicted, y_test,squared=True)))
        errors_ml.append(np.sqrt(2*mean_squared_error(y_ML_old, y_predicted,squared=True)))
    return errors, errors_ml, alphas
    
    
    